# [IEMCow](https://mesonet.agron.iastate.edu/cow/): 2022 warnings, by zone

#### Import Python tools

In [2]:
%load_ext lab_black

In [3]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_stiles as altstiles
import numpy as np
import requests
from pathlib import Path

In [4]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [5]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 10000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [6]:
today = pd.to_datetime("today").strftime("%Y-%m-%d")

---

# Lookup table for NWS zones to home counties

#### Read [zone-county correlation file](https://www.weather.gov/gis/ZoneCounty)

In [7]:
zones = pd.read_csv(
    "data/raw/nws_zone_to_county_lookup.txt",
    sep="|",
    header=0,
    names=[
        "state",
        "zone",
        "cwa",
        "name",
        "state_zone",
        "county",
        "fips",
        "time_zone",
        "fe_area",
        "lat",
        "lon",
    ],
)

---

## Get warnings archive

In [8]:
archive_url = "https://mesonet.agron.iastate.edu/pickup/wwa/2022_all.zip"

In [9]:
gdf_src = gpd.read_file(archive_url)

In [10]:
# gdf_src = gpd.read_file("data/raw/2022_all/wwa_202201010000_202212312359.shp")

In [11]:
gdf_src.columns = gdf_src.columns.str.lower()

---

#### Just the geography

In [12]:
zones_gdf = gpd.read_file("data/raw/public_forecast_zones/z_07se22.shp")

In [13]:
zones_gdf.columns = zones_gdf.columns.str.lower()

In [14]:
zones_gdf.head()

,state,cwa,time_zone,fe_area,zone,name,state_zone,lon,lat,shortname,geometry
0,AL,BMX,C,ec,019,Calhoun,AL019,-85.8261,33.7714,Calhoun,"POLYGON ((-85.53010 33.94181, -85.53200 33.897..."
1,AL,MOB,C,sc,057,Butler,AL057,-86.6803,31.7524,Butler,"POLYGON ((-86.44800 31.96431, -86.44769 31.952..."
2,AL,BMX,C,se,046,Bullock,AL046,-85.7161,32.1005,Bullock,"POLYGON ((-85.43349 32.23421, -85.43120 32.198..."
3,AL,BMX,C,cc,017,Blount,AL017,-86.5674,33.9809,Blount,"POLYGON ((-86.45229 34.25951, -86.44420 34.257..."
4,AL,BMX,C,cc,034,Bibb,AL034,-87.1264,32.9986,Bibb,"POLYGON ((-87.02679 33.24601, -87.02599 33.231..."


#### Just the warning data related to heat

In [15]:
sig = {
    "W": "Warning",
    "Y": "Advisory",
    "A": "Watch",
}

In [16]:
warnings_df_src = gdf_src[gdf_src["phenom"].fillna("").str.contains("EH")][
    [
        "issued",
        "expired",
        "phenom",
        "gtype",
        "sig",
        "etn",
        "status",
        "nws_ugc",
    ]
].copy()

In [17]:
# ext_heat_warn = warnings_df_src[(warnings_df_src["sig"] == "W")]
ext_heat_warn = warnings_df_src.copy()

---

## Aggregate

In [18]:
heat_grouped = (
    ext_heat_warn.groupby(["nws_ugc"])
    .agg({"phenom": "count"})
    .sort_values("phenom", ascending=False)
    .reset_index()
    .rename(columns={"phenom": "count"})
)

In [19]:
heat_grouped["state_zone"] = heat_grouped["nws_ugc"].str.replace("Z", "")

In [20]:
heat_grouped.head()

,nws_ugc,count,state_zone
0,CAZ199,16,CA199
1,CAZ198,16,CA198
2,NVZ017,15,NV017
3,CAZ522,14,CA522
4,CAZ060,14,CA060


In [21]:
zones_gdf.head()

,state,cwa,time_zone,fe_area,zone,name,state_zone,lon,lat,shortname,geometry
0,AL,BMX,C,ec,019,Calhoun,AL019,-85.8261,33.7714,Calhoun,"POLYGON ((-85.53010 33.94181, -85.53200 33.897..."
1,AL,MOB,C,sc,057,Butler,AL057,-86.6803,31.7524,Butler,"POLYGON ((-86.44800 31.96431, -86.44769 31.952..."
2,AL,BMX,C,se,046,Bullock,AL046,-85.7161,32.1005,Bullock,"POLYGON ((-85.43349 32.23421, -85.43120 32.198..."
3,AL,BMX,C,cc,017,Blount,AL017,-86.5674,33.9809,Blount,"POLYGON ((-86.45229 34.25951, -86.44420 34.257..."
4,AL,BMX,C,cc,034,Bibb,AL034,-87.1264,32.9986,Bibb,"POLYGON ((-87.02679 33.24601, -87.02599 33.231..."


---

## Merge

In [22]:
heat_gdf = gpd.GeoDataFrame(
    pd.merge(heat_grouped, zones_gdf, on="state_zone", how="right")
)

In [23]:
heat_gdf["count"] = heat_gdf["count"].fillna(0)

In [24]:
states = [
    "AL",
    "AR",
    "KS",
    "KY",
    "TN",
    "TX",
    "CO",
    "ID",
    "MA",
    "MT",
    "AZ",
    "WA",
    "MN",
    "WY",
    "NY",
    "CT",
    "NJ",
    "VA",
    "MD",
    "AS",
    "DE",
    "GA",
    "IA",
    "IL",
    "IN",
    "WV",
    "OR",
    "SD",
    "UT",
    "MI",
    "MO",
    "MS",
    "NC",
    "ND",
    "NE",
    "VT",
    "NH",
    "NV",
    "OH",
    "OK",
    "PA",
    "SC",
    "VI",
    "WI",
    "LA",
    "FL",
    "ME",
    "CA",
    "NM",
    "RI",
    "DC",
]

In [25]:
heat_gdf = heat_gdf[heat_gdf["state"].isin(states)]

---

## Export

In [26]:
heat_gdf[["state_zone", "count", "name", "state", "geometry"]].to_file(
    "data/raw/excessive_heat_warnings_2022.geojson", driver="GeoJSON"
)

In [27]:
heat_gdf.head()

,nws_ugc,count,state_zone,state,cwa,time_zone,fe_area,zone,name,lon,lat,shortname,geometry
0,NaN,0.0,AL019,AL,BMX,C,ec,019,Calhoun,-85.8261,33.7714,Calhoun,"POLYGON ((-85.53010 33.94181, -85.53200 33.897..."
1,NaN,0.0,AL057,AL,MOB,C,sc,057,Butler,-86.6803,31.7524,Butler,"POLYGON ((-86.44800 31.96431, -86.44769 31.952..."
2,NaN,0.0,AL046,AL,BMX,C,se,046,Bullock,-85.7161,32.1005,Bullock,"POLYGON ((-85.43349 32.23421, -85.43120 32.198..."
3,NaN,0.0,AL017,AL,BMX,C,cc,017,Blount,-86.5674,33.9809,Blount,"POLYGON ((-86.45229 34.25951, -86.44420 34.257..."
4,NaN,0.0,AL034,AL,BMX,C,cc,034,Bibb,-87.1264,32.9986,Bibb,"POLYGON ((-87.02679 33.24601, -87.02599 33.231..."


In [28]:
!mapshaper -i data/raw/excessive_heat_warnings_2022.geojson  -simplify percentage=.1 no-repair  -o data/processed/excessive_heat_warnings_2022_simple.geojson

[o] Wrote data/processed/excessive_heat_warnings_2022_simple.geojson


In [29]:
heat_gdf[
    [
        "nws_ugc",
        "count",
        "state_zone",
        "state",
        "cwa",
        "time_zone",
        "fe_area",
        "zone",
        "name",
        "lon",
        "lat",
        "shortname",
    ]
].to_csv("data/processed/excessive_heat_warnings_2022.csv", index=False)